In [1]:
import openpyxl
import pandas as pd
from datetime import datetime
from datetime import date
import pandas as pd
import pandas_gbq as pgbq
from time import strftime
import datetime
import numpy as np
from dateutil.relativedelta import relativedelta

project_id = 'tensile-oarlock-191715'

forecast_flag = 1 # 1 for forecast AFA and Revenue numbers, 0 for actual finance numbers
test = True # True stops snapshot file from updating

# The current reporting month end
# Modify as needed when new script is run

# The name of the CSV output file. Eventually, this should be commented out.
# Modify as needed when new script is run
csv_out_name = 'test_coverme_finance_view_2022_07_31_mst_2.csv'

# This is the output table that we write to in MetaBase
# Do not modify this
out_table = 'risk.CM_finance_view_alt'
#out_table = 'risk.CM_finance_view_dl'


if(forecast_flag== 0):
    revenue = input("Enter revenue number:")
    AFA = input("Enter AFA number:")
    

next_month = datetime.date.today().replace(day=28) + datetime.timedelta(days=4)
last_day = next_month - datetime.timedelta(days=next_month.day) - relativedelta(months=1)
last_month =  datetime.datetime.strptime(str(last_day - relativedelta(months=1)), '%Y-%m-%d')
curr_month_end = last_day 
periods = pd.date_range(start='2021-08-15', 
                       end=str(last_day),
                       freq='M') # W for weekly M for monthly

today = date.today()
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

periods

DatetimeIndex(['2021-08-31', '2021-09-30', '2021-10-31', '2021-11-30',
               '2021-12-31', '2022-01-31', '2022-02-28', '2022-03-31',
               '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31',
               '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30',
               '2022-12-31'],
              dtype='datetime64[ns]', freq='M')

In [2]:
def loans_by_period(start, end):
    
    query = """with
    
                mapping as (select 
                                distinct account_group_identifier, 
                                    user_reference 
                                    from accounts.kledger_accounts group by 1,2),
                                    
                transactions as (select 
                    b.user_reference,
                    a.UserAccount,
                    a.TransactionID,
                    date(PostedAt, 'America/Denver') as PostedAt,
                    last_day(date(a.PostedAt, 'America/Denver'), MONTH) as t_month,
                    case
                        when a.PaymentCode in ('ODR-DR-A-E-001') then '1. fee'
                        when a.PaymentCode in ('ODR-CR-A-E-001') then '2. loan'
                        else '3. repayment'
                    end as TType,
                    case 
                        when a.PaymentCode in ('ODR-DR-A-E-002', 'ODR-DR-A-E-001') then -1*cast(trim(a.Amount,"$") as FLOAT64)
                        else cast(trim(a.Amount,"$") as FLOAT64)
                    end as Amount
                    from kohoapi.transaction_succeeded_events as a
                        left join mapping as b on a.UserAccount = b.account_group_identifier
                    where PaymentCode in ('ODR-DR-A-E-001','ODR-CR-A-E-001','ODR-DR-A-E-002') and date(PostedAt, 'America/Denver') <= '{1}'),
                
                last_loan as 
                    (select 
                        user_reference, 
                        max(PostedAt) as last_loan_date 
                    from transactions where TType = '2. loan' group by 1),
                
                -- need this to make sure that we don't somehow have to deal with an error where 
                -- a re-payment is the very first transaction
                first_loan as 
                    (select 
                        user_reference, 
                        min(PostedAt) as first_loan_date 
                    from transactions where TType = '2. loan' group by 1),
                
                loans_in_month as 
                    (select 
                        distinct user_reference, 
                        1 as loan, 
                        sum(Amount) as loaned_in_month 
                    from transactions where TType = '2. loan' AND t_month = '{1}' group by 1),
                
                repayments_in_month as 
                    (select 
                        distinct user_reference, 
                        count(*) as num_repayment, 
                        sum(Amount) as repaid_in_month 
                            from transactions where TType = '3. repayment' AND t_month = '{1}' group by 1),
                
                OS as 
                    (select 
                        distinct a.user_reference, 
                        sum(a.Amount) as OS 
                    from transactions as a 
                    inner join first_loan as b 
                        on a.user_reference = b.user_reference 
                            and a.PostedAt >= b.first_loan_date where TType in ('2. loan', '3. repayment') group by 1),
                
                last_fee as 
                    (select 
                        user_reference, 
                        max(PostedAt) as last_fee_date 
                    from transactions where TType = '1. fee' group by 1),
                    
                last_payment as
                    (select 
                        user_reference, 
                        max(PostedAt) as last_payment_date 
                    from transactions where TType = '3. repayment' group by 1),
                
                -- counting fee payments after last disbursal
                
                count_fee as 
                    (select 
                        distinct a.user_reference, 
                        count(a.TType) as num_fee_payments 
                    from transactions as a 
                    inner join last_loan as b 
                        on a.user_reference = b.user_reference 
                            and date(a.PostedAt) > date(last_loan_date) where TType = '1. fee' group by 1),
                        
                -- counting fee payments that were SUPPOSED to be made fater disbursal
                        
                count_fee_s as 
                    (select 
                        user_reference, 
                        last_loan_date, 
                        date_diff(date('{1}'),date(last_loan_date), DAY)/30 as diff from last_loan),
                
                final as (select date('{1}') as snapshot, 
                            a.user_reference,
                            b.OS,
                            c.last_loan_date,
                            d.last_fee_date,
                            i.last_payment_date,
                            IFNULL(g.num_fee_payments,0) as num_fee_payments,
                            floor(h.diff) as num_expected_fee_payments,
                            date_add(date(c.last_loan_date), INTERVAL 30 DAY) as orig_due_date,
                            case
                                when date(c.last_loan_date) <= '2022-05-01' then date_add(date(c.last_loan_date), INTERVAL 30 DAY)
                                
                                -- Need to add IFNULL on the date as somehow the last fee is not captured
                                
                                when date(c.last_loan_date) > '2022-05-01' AND IFNULL(g.num_fee_payments,0) >= floor(h.diff) then date_add(date(IFNULL(d.last_fee_date, c.last_loan_date)), INTERVAL 30 DAY)
                                when date(c.last_loan_date) > '2022-05-01' AND IFNULL(g.num_fee_payments,0) < floor(h.diff) then date_add(date(c.last_loan_date), INTERVAL ((1+IFNULL(g.num_fee_payments,0))*30) DAY)
                            end as due_date,
                            
                            
                            case
                                when i.last_payment_date IS NOT NULL AND date(i.last_payment_date) >= date(c.last_loan_date) then date(i.last_payment_date)
                                else date(c.last_loan_date)   
                            end as update_date,
                            
                            e.loan as num_loans_in_month,
                            e.loaned_in_month as total_loaned_in_month,
                            f.num_repayment as num_repayments_in_month,
                            -f.repaid_in_month as total_repaid_in_month
                            from (select distinct user_reference from transactions group by 1) as a
                            left join OS as b on a.user_reference = b.user_reference
                            left join last_loan as c on a.user_reference = c.user_reference
                            left join last_fee as d on a.user_reference = d.user_reference
                            left join loans_in_month as e on a.user_reference = e.user_reference
                            left join repayments_in_month as f on a.user_reference = f.user_reference
                            left join count_fee as g on a.user_reference = g.user_reference
                            left join count_fee_s as h on a.user_reference = h.user_reference
                            left join last_payment as i on a.user_reference = i.user_reference)
 -- Here is where we calculate the delinquency buckets                           
 select 
    *,
    last_day(date_sub(date('{1}'), INTERVAL 1 MONTH), MONTH) as prev_snap,
    last_day(date(last_loan_date), MONTH) as cohort_end,
    last_day(date(last_payment_date), MONTH) as cohort_paid,
    case
        when OS = 0 then "1. Inactive"
        when OS > 0 AND date(due_date) >= date('{1}') then "2. current"
        when OS > 0 and date_diff(date('{1}'),date(due_date),DAY) between 1 and 30 then '3. 1 to 30'
        when OS > 0 and date_diff(date('{1}'),date(due_date),DAY) between 31 and 60 then '4. 31 to 60'
        when OS > 0 and date_diff(date('{1}'),date(due_date),DAY) between 61 and 90 then '5. 61 to 90'
        when OS > 0 and date_diff(date('{1}'),date(due_date),DAY) > 90 then '6. 91+'
        else "7. Balance Issue"
    end as snapshot_status
    
-- Have to investigate why some OS is coming up NULL    
 from final where OS IS NOT NULL""".format(start, end)

    loans = pgbq.read_gbq(query, project_id, dialect='standard')
    
    return loans

# initializing curr_loans
curr_loans = loans_by_period(periods[0].strftime('%Y-%m-%d'),periods[periods.size-1].strftime('%Y-%m-%d'))

i = 1
while i < periods.size-1:
    curr_loans = pd.concat([curr_loans, loans_by_period(periods[0].strftime('%Y-%m-%d'),periods[i].strftime('%Y-%m-%d'))], ignore_index=True)
    i += 1

Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|


In [3]:
for i in range(len(curr_loans.columns)):
    if(curr_loans.dtypes.values[i] == "dbdate"):
        curr_loans[curr_loans.columns[i]]= pd.to_datetime(curr_loans[curr_loans.columns[i]])
        
curr_month = datetime.datetime.strptime(str(curr_month_end), '%Y-%m-%d')
#curr_month = curr_month_end

In [4]:
# output the results to a spreadsheet
# Delete snapshots that are greater than the current month
curr_loans = curr_loans[curr_loans.snapshot <= curr_month]
curr_loans = curr_loans.drop_duplicates()
# curr_loans.to_csv('new_pay_logic_test_2.csv', index=False)

In [5]:
curr_loans['days_between_paid_and_due'] = ((curr_loans['update_date'] - curr_loans['due_date'])/np.timedelta64(1,'D')).astype(int)

In [6]:
curr_loans.prev_snap= pd.to_datetime(curr_loans.prev_snap)


In [7]:
# Self Merge to get previous status

curr_loans_copy = curr_loans[['snapshot', 'user_reference', 'snapshot_status']]
curr_loans = curr_loans.merge(curr_loans_copy, how='left', left_on=["user_reference", "prev_snap"], right_on=["user_reference","snapshot"], suffixes=('_left', '_right'))
curr_loans.drop(['snapshot_right'],axis=1,inplace=True)
curr_loans = curr_loans.rename({'snapshot_left': 'snapshot', 'snapshot_status_left': 'snapshot_status','snapshot_status_right' :  'prev_status'}, axis='columns')
curr_loans['prev_status'].fillna('0. non existent', inplace = True)
curr_loans.head()

,snapshot,user_reference,OS,last_loan_date,last_fee_date,last_payment_date,num_fee_payments,num_expected_fee_payments,orig_due_date,due_date,...,num_loans_in_month,total_loaned_in_month,num_repayments_in_month,total_repaid_in_month,prev_snap,cohort_end,cohort_paid,snapshot_status,days_between_paid_and_due,prev_status
0,2022-12-31,C4mjTPiM,50.00,2022-07-26,2022-12-25,2022-07-19,6,5.0,2022-08-25,2023-01-24,...,<NA>,NaN,<NA>,NaN,2022-11-30,2022-07-31,2022-07-31,2. current,-182,2. current
1,2022-12-31,SBgDGaGP,49.56,2022-11-27,2022-11-27,2022-12-29,0,1.0,2022-12-27,2022-12-27,...,<NA>,NaN,1,0.44,2022-11-30,2022-11-30,2022-12-31,3. 1 to 30,2,2. current
2,2022-12-31,TFTtdU6q,50.00,2022-06-10,2022-12-26,2022-05-27,7,6.0,2022-07-10,2023-01-25,...,<NA>,NaN,<NA>,NaN,2022-11-30,2022-06-30,2022-05-31,2. current,-229,2. current
3,2022-12-31,UmdaqHID,50.00,2022-06-10,2022-12-29,2022-05-27,7,6.0,2022-07-10,2023-01-28,...,<NA>,NaN,<NA>,NaN,2022-11-30,2022-06-30,2022-05-31,2. current,-232,2. current
4,2022-12-31,nOdvE1k7,50.00,2022-06-02,2022-12-01,2022-06-02,6,7.0,2022-07-02,2022-12-29,...,<NA>,NaN,<NA>,NaN,2022-11-30,2022-06-30,2022-06-30,3. 1 to 30,-210,3. 1 to 30


In [8]:
curr_loans_2 = curr_loans.copy(deep=True)

In [9]:
def get_new(row):
    if row['snapshot'] == row['cohort_end']:
        return 'Yes'
    else:
        return 'No'

In [10]:
def get_paid_class(row):
    if row['snapshot_status'] == '1. Inactive':
        if row['days_between_paid_and_due'] <= 0:
            return '1. paid on time'
        elif 0 < row['days_between_paid_and_due'] < 91:
            return '2. paid late'
        elif row['prev_status'] == '6. 91+':
            return '3. paid after default'
        else:
            return '2. paid late'
    else: 
        return '4. NA'

In [11]:
def get_default(row):
    if row['snapshot_status'] == '6. 91+':
        if row['prev_status'] != '6. 91+':
            return '1. Yes'
        else:
            return '2. No'
    else:
        return '3. NA'

In [12]:
def get_paid_cohort(row):
    if row['snapshot_status'] == '1. Inactive':
        if row['cohort_paid'] == row['snapshot']:
            return '1. Yes'
        else:
            return '2. No'
    else:
            return '3. NA'

In [13]:
curr_loans_2['new_loan'] = curr_loans_2.apply(get_new, axis=1)
curr_loans_2['paid_class'] = curr_loans_2.apply(get_paid_class, axis=1)
curr_loans_2['new_default'] = curr_loans_2.apply(get_default, axis=1)
curr_loans_2['paid_in_cohort'] = curr_loans_2.apply(get_paid_cohort, axis=1)
curr_loans_2.head()

,snapshot,user_reference,OS,last_loan_date,last_fee_date,last_payment_date,num_fee_payments,num_expected_fee_payments,orig_due_date,due_date,...,prev_snap,cohort_end,cohort_paid,snapshot_status,days_between_paid_and_due,prev_status,new_loan,paid_class,new_default,paid_in_cohort
0,2022-12-31,C4mjTPiM,50.00,2022-07-26,2022-12-25,2022-07-19,6,5.0,2022-08-25,2023-01-24,...,2022-11-30,2022-07-31,2022-07-31,2. current,-182,2. current,No,4. NA,3. NA,3. NA
1,2022-12-31,SBgDGaGP,49.56,2022-11-27,2022-11-27,2022-12-29,0,1.0,2022-12-27,2022-12-27,...,2022-11-30,2022-11-30,2022-12-31,3. 1 to 30,2,2. current,No,4. NA,3. NA,3. NA
2,2022-12-31,TFTtdU6q,50.00,2022-06-10,2022-12-26,2022-05-27,7,6.0,2022-07-10,2023-01-25,...,2022-11-30,2022-06-30,2022-05-31,2. current,-229,2. current,No,4. NA,3. NA,3. NA
3,2022-12-31,UmdaqHID,50.00,2022-06-10,2022-12-29,2022-05-27,7,6.0,2022-07-10,2023-01-28,...,2022-11-30,2022-06-30,2022-05-31,2. current,-232,2. current,No,4. NA,3. NA,3. NA
4,2022-12-31,nOdvE1k7,50.00,2022-06-02,2022-12-01,2022-06-02,6,7.0,2022-07-02,2022-12-29,...,2022-11-30,2022-06-30,2022-06-30,3. 1 to 30,-210,3. 1 to 30,No,4. NA,3. NA,3. NA


In [14]:
# Delete snapshots that are greater than the current month
final = curr_loans_2[curr_loans_2.snapshot <= curr_month]


# Sort
final.sort_values(['user_reference', 'snapshot'], inplace = True)        
            
# drop columns
# final.drop(labels=['loan_type','status','prev_snap','days_between_curr_and_dep','days_between_paid_and_due','days_between_curr_and_paid','days_between_curr_and_due'],axis=1, inplace=True)
final.drop(labels=['days_between_paid_and_due'],axis=1, inplace=True)

# delete extra paid status rows
# final.drop(final[(final.snapshot_status == '6. paid') and (final.snapshot_status == final.snapshot_status)].index, inplace=True)
final_dedup = final.drop_duplicates()

# reset index
final.reset_index(drop=True, inplace=True)

# output the results to a spreadsheet
final_dedup.to_csv(csv_out_name, index=False)

# Upload data back to the cloud 
if not test:
    pgbq.to_gbq(final_dedup, out_table, project_id, if_exists='replace')

Reading current version of the file

In [40]:
wb = openpyxl.load_workbook("Monthly Credit Risk Reporting.xlsx",data_only=True)
Old_Cover_df = pd.DataFrame(wb['Cover'].values)

In [41]:
Old_Cover_df = Old_Cover_df.dropna(axis = 1,how='all')
Old_Cover_df = Old_Cover_df.dropna(how='all')
index_df = pd.read_csv("Cover index.csv")
Old_Cover_df.columns = Old_Cover_df.iloc[0].astype('string')
Old_Cover_df = Old_Cover_df.drop(0).reset_index()
Old_Cover_df = Old_Cover_df.join(index_df)
Old_Cover_df= Old_Cover_df.set_index('Row Name')

In [42]:
recent_df = Old_Cover_df.loc[:,str((curr_month - relativedelta(months=6)).replace(day=1)):str((curr_month - relativedelta(months=1)).replace(day=1))]
recent_df[str(curr_month.replace(day=1))] = ' '

Replicating SQL code used to summarize data

In [43]:
status_df = final_dedup.groupby(["snapshot","snapshot_status"]).agg({'user_reference':'nunique','OS':'sum'})
status_df = status_df.loc[curr_month]

def tobbs(df,prefix):
    if('current' in df['snapshot_status']):
        return prefix + 'Current'
    elif('to' in df['snapshot_status']):
        return prefix + df['snapshot_status'][df['snapshot_status'].find(' ')+1:df['snapshot_status'].find(' ',df['snapshot_status'].find(' ')+1)] + '-' + df['snapshot_status'][-2:]
    elif('+' in df['snapshot_status']):
        return prefix + 'Cumulative 91+'
status_df = status_df.reset_index()
status_df['snapshot_status_balance'] = status_df.apply(lambda x: tobbs(x,'Total Outstanding Balance by Status '),axis = 1)
status_df['snapshot_status_#'] = status_df.apply(lambda x: tobbs(x,'Total # of Outstanding Loans by Status '),axis = 1)
temp_df = status_df[pd.notna(status_df['snapshot_status_balance'])].reset_index()

for i in range(len(temp_df)):
    recent_df.loc[temp_df.loc[i,'snapshot_status_balance'],str(curr_month.replace(day=1))] = temp_df.loc[i,'OS']
    recent_df.loc[temp_df.loc[i,'snapshot_status_#'],str(curr_month.replace(day=1))] = temp_df.loc[i,'user_reference'] 

In [44]:
temp_df = final_dedup[final_dedup['snapshot_status'] == '6. 91+']
temp_df = temp_df[temp_df['new_default'] == '1. Yes']
ninetyoneplus_status_df = temp_df.groupby(["snapshot","snapshot_status"]).agg({'user_reference':'nunique','OS':'sum'})
ninetyoneplus_status_df = ninetyoneplus_status_df.loc[curr_month]

recent_df.loc['Total Outstanding Balance by Status 91-120',str(curr_month.replace(day=1))] = ninetyoneplus_status_df.loc['6. 91+','OS']
recent_df.loc['Total # of Outstanding Loans by Status 91-120',str(curr_month.replace(day=1))] = ninetyoneplus_status_df.loc['6. 91+','user_reference']

In [45]:
recent_df.loc['Total Outstanding Balance by Status Net write-off in the month',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status Cumulative 91+',str(curr_month.replace(day=1))] - \
recent_df.loc['Total Outstanding Balance by Status Cumulative 91+',str(last_month.replace(day=1))]

recent_df.loc['Total # of Outstanding Loans by Status Net write-off in the month',str(curr_month.replace(day=1))] = \
recent_df.loc['Total # of Outstanding Loans by Status Cumulative 91+',str(curr_month.replace(day=1))] - \
recent_df.loc['Total # of Outstanding Loans by Status Cumulative 91+',str(last_month.replace(day=1))]

recent_df.loc['Post Write Off Recoveries',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status 91-120',str(curr_month.replace(day=1))] - \
recent_df.loc['Total Outstanding Balance by Status Net write-off in the month',str(curr_month.replace(day=1))]

recent_df.loc['Recovery Rate',str(curr_month.replace(day=1))] = \
recent_df.loc['Post Write Off Recoveries',str(curr_month.replace(day=1))] / \
recent_df.loc['Total Outstanding Balance by Status Cumulative 91+',str(last_month.replace(day=1))]

In [46]:
temp_df = final_dedup[final_dedup['new_loan'] == 'Yes']
origination_df = temp_df.groupby("snapshot").agg({'user_reference':'nunique','total_loaned_in_month':'sum'})
origination_df = origination_df.loc[curr_month]

recent_df.loc['(Capital Deployed in month) Total Amount of New Loan Issued ',str(curr_month.replace(day=1))] = origination_df['total_loaned_in_month']
recent_df.loc['Total # of New Loan Issued',str(curr_month.replace(day=1))] = origination_df['user_reference']

In [47]:
#Loan Origination
recent_df.loc['Avg Amount/Loan',str(curr_month.replace(day=1))] = \
recent_df.loc['(Capital Deployed in month) Total Amount of New Loan Issued ',str(curr_month.replace(day=1))] / \
recent_df.loc['Total # of New Loan Issued',str(curr_month.replace(day=1))]

In [48]:
#Dollar Roll Rates
recent_df.loc['Dollar Roll Rates Current to 1-30',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status 1-30',str(curr_month.replace(day=1))] \
/ recent_df.loc['Total Outstanding Balance by Status Current',str(last_month.replace(day=1))]

recent_df.loc['Dollar Roll Rates 1-30 to 31-60',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status 31-60',str(curr_month.replace(day=1))] \
/ recent_df.loc['Total Outstanding Balance by Status 1-30',str(last_month.replace(day=1))]

recent_df.loc['Dollar Roll Rates 31-60 to 61-90',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status 61-90',str(curr_month.replace(day=1))] \
/ recent_df.loc['Total Outstanding Balance by Status 31-60',str(last_month.replace(day=1))]

recent_df.loc['Dollar Roll Rates 61-90 to 91+',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status 91-120',str(curr_month.replace(day=1))] \
/ recent_df.loc['Total Outstanding Balance by Status 61-90',str(last_month.replace(day=1))]

In [49]:
#Unit Roll Rate
recent_df.loc['Unit Roll Rates Current to 1-30',str(curr_month.replace(day=1))] = \
recent_df.loc['Total # of Outstanding Loans by Status 1-30',str(curr_month.replace(day=1))] \
/ recent_df.loc['Total # of Outstanding Loans by Status Current',str(last_month.replace(day=1))]

recent_df.loc['Unit Roll Rates 1-30 to 31-60',str(curr_month.replace(day=1))] = \
recent_df.loc['Total # of Outstanding Loans by Status 31-60',str(curr_month.replace(day=1))] \
/ recent_df.loc['Total # of Outstanding Loans by Status 1-30',str(last_month.replace(day=1))]

recent_df.loc['Unit Roll Rates 31-60 to 61-90',str(curr_month.replace(day=1))] = \
recent_df.loc['Total # of Outstanding Loans by Status 61-90',str(curr_month.replace(day=1))] \
/ recent_df.loc['Total # of Outstanding Loans by Status 31-60',str(last_month.replace(day=1))]

recent_df.loc['Unit Roll Rates 61-90 to 91+',str(curr_month.replace(day=1))] = \
recent_df.loc['Total # of Outstanding Loans by Status 91-120',str(curr_month.replace(day=1))] \
/ recent_df.loc['Total # of Outstanding Loans by Status 61-90',str(last_month.replace(day=1))]

In [50]:
#PD (Probability of Default)
recent_df.loc['PD (Probability of Default) Current',str(curr_month.replace(day=1))] = \
recent_df.loc['Dollar Roll Rates Current to 1-30',str(curr_month.replace(day=1))] * \
recent_df.loc['Dollar Roll Rates 1-30 to 31-60',str(curr_month.replace(day=1))] * \
recent_df.loc['Dollar Roll Rates 31-60 to 61-90',str(curr_month.replace(day=1))] * \
recent_df.loc['Dollar Roll Rates 61-90 to 91+',str(curr_month.replace(day=1))]

recent_df.loc['PD (Probability of Default) 1-30',str(curr_month.replace(day=1))] = \
recent_df.loc['Dollar Roll Rates 1-30 to 31-60',str(curr_month.replace(day=1))] * \
recent_df.loc['Dollar Roll Rates 31-60 to 61-90',str(curr_month.replace(day=1))] * \
recent_df.loc['Dollar Roll Rates 61-90 to 91+',str(curr_month.replace(day=1))]

recent_df.loc['PD (Probability of Default) 31-60',str(curr_month.replace(day=1))] = \
recent_df.loc['Dollar Roll Rates 31-60 to 61-90',str(curr_month.replace(day=1))] * \
recent_df.loc['Dollar Roll Rates 61-90 to 91+',str(curr_month.replace(day=1))]

recent_df.loc['PD (Probability of Default) 61-90',str(curr_month.replace(day=1))] = \
recent_df.loc['Dollar Roll Rates 61-90 to 91+',str(curr_month.replace(day=1))]

In [51]:
#Avg. PD (Probability of Default)
recent_df.loc['Avg. PD (Probability of Default) Current',str(curr_month.replace(day=1))] = (\
recent_df.loc['PD (Probability of Default) Current',str(curr_month.replace(day=1))] + \
recent_df.loc['PD (Probability of Default) Current',str(datetime.datetime.strptime(str(last_day - relativedelta(months=1)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) Current',str(datetime.datetime.strptime(str(last_day - relativedelta(months=2)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) Current',str(datetime.datetime.strptime(str(last_day - relativedelta(months=3)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) Current',str(datetime.datetime.strptime(str(last_day - relativedelta(months=4)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) Current',str(datetime.datetime.strptime(str(last_day - relativedelta(months=5)), '%Y-%m-%d').replace(day=1))] \
)/6

recent_df.loc['Avg. PD (Probability of Default) 1-30',str(curr_month.replace(day=1))] = (\
recent_df.loc['PD (Probability of Default) 1-30',str(curr_month.replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 1-30',str(datetime.datetime.strptime(str(last_day - relativedelta(months=1)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 1-30',str(datetime.datetime.strptime(str(last_day - relativedelta(months=2)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 1-30',str(datetime.datetime.strptime(str(last_day - relativedelta(months=3)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 1-30',str(datetime.datetime.strptime(str(last_day - relativedelta(months=4)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 1-30',str(datetime.datetime.strptime(str(last_day - relativedelta(months=5)), '%Y-%m-%d').replace(day=1))] \
)/6

recent_df.loc['Avg. PD (Probability of Default) 31-60',str(curr_month.replace(day=1))] = (\
recent_df.loc['PD (Probability of Default) 31-60',str(curr_month.replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 31-60',str(datetime.datetime.strptime(str(last_day - relativedelta(months=1)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 31-60',str(datetime.datetime.strptime(str(last_day - relativedelta(months=2)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 31-60',str(datetime.datetime.strptime(str(last_day - relativedelta(months=3)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 31-60',str(datetime.datetime.strptime(str(last_day - relativedelta(months=4)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 31-60',str(datetime.datetime.strptime(str(last_day - relativedelta(months=5)), '%Y-%m-%d').replace(day=1))] \
)/6

recent_df.loc['Avg. PD (Probability of Default) 61-90',str(curr_month.replace(day=1))] = (\
recent_df.loc['PD (Probability of Default) 61-90',str(curr_month.replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 61-90',str(datetime.datetime.strptime(str(last_day - relativedelta(months=1)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 61-90',str(datetime.datetime.strptime(str(last_day - relativedelta(months=2)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 61-90',str(datetime.datetime.strptime(str(last_day - relativedelta(months=3)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 61-90',str(datetime.datetime.strptime(str(last_day - relativedelta(months=4)), '%Y-%m-%d').replace(day=1))] + \
recent_df.loc['PD (Probability of Default) 61-90',str(datetime.datetime.strptime(str(last_day - relativedelta(months=5)), '%Y-%m-%d').replace(day=1))] \
)/6

In [52]:
#PD used in Provision Calculation
recent_df.loc['PD used in Provision Calculation Current',str(curr_month.replace(day=1))] = \
recent_df.loc['Avg. PD (Probability of Default) Current',str(curr_month.replace(day=1))]

recent_df.loc['PD used in Provision Calculation 1-30',str(curr_month.replace(day=1))] = \
recent_df.loc['Avg. PD (Probability of Default) 1-30',str(curr_month.replace(day=1))]

recent_df.loc['PD used in Provision Calculation 31-60',str(curr_month.replace(day=1))] = \
recent_df.loc['Avg. PD (Probability of Default) 31-60',str(curr_month.replace(day=1))]

recent_df.loc['PD used in Provision Calculation 61-90',str(curr_month.replace(day=1))] = \
recent_df.loc['Avg. PD (Probability of Default) 61-90',str(curr_month.replace(day=1))]

In [53]:
#Provision by Bucket
recent_df.loc['Provision by Bucket Current',str(curr_month.replace(day=1))] = \
recent_df.loc['PD used in Provision Calculation Current',str(curr_month.replace(day=1))] * \
recent_df.loc['Total Outstanding Balance by Status Current',str(curr_month.replace(day=1))]

recent_df.loc['Provision by Bucket 1-30',str(curr_month.replace(day=1))] = \
recent_df.loc['PD used in Provision Calculation 1-30',str(curr_month.replace(day=1))] * \
recent_df.loc['Total Outstanding Balance by Status 1-30',str(curr_month.replace(day=1))]

recent_df.loc['Provision by Bucket 31-60',str(curr_month.replace(day=1))] = \
recent_df.loc['PD used in Provision Calculation 31-60',str(curr_month.replace(day=1))] * \
recent_df.loc['Total Outstanding Balance by Status 31-60',str(curr_month.replace(day=1))]

recent_df.loc['Provision by Bucket 61-90',str(curr_month.replace(day=1))] = \
recent_df.loc['PD used in Provision Calculation 61-90',str(curr_month.replace(day=1))] * \
recent_df.loc['Total Outstanding Balance by Status 61-90',str(curr_month.replace(day=1))]

recent_df.loc['Total',str(curr_month.replace(day=1))] = \
recent_df.loc['Provision by Bucket Current',str(curr_month.replace(day=1))] + \
recent_df.loc['Provision by Bucket 1-30',str(curr_month.replace(day=1))] + \
recent_df.loc['Provision by Bucket 31-60',str(curr_month.replace(day=1))] + \
recent_df.loc['Provision by Bucket 61-90',str(curr_month.replace(day=1))]

In [54]:
recent_df.loc['Total Provision ',str(curr_month.replace(day=1))] = \
recent_df.loc['Total',str(curr_month.replace(day=1))]

recent_df.loc['Provision Change',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Provision ',str(curr_month.replace(day=1))] - \
recent_df.loc['Total Provision ',str(last_month.replace(day=1))]

recent_df.loc['Total cumulative writeoff',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status Cumulative 91+',str(curr_month.replace(day=1))]

recent_df.loc['Write off Change',str(curr_month.replace(day=1))] = \
recent_df.loc['Total cumulative writeoff',str(curr_month.replace(day=1))] - \
recent_df.loc['Total cumulative writeoff',str(last_month.replace(day=1))]

In [55]:
# Average Loan Size by Status
recent_df.loc['Average Loan Size by Status Current',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status Current',str(curr_month.replace(day=1))] / \
recent_df.loc['Total # of Outstanding Loans by Status Current',str(curr_month.replace(day=1))]

recent_df.loc['Average Loan Size by Status 1-30',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status 1-30',str(curr_month.replace(day=1))] / \
recent_df.loc['Total # of Outstanding Loans by Status 1-30',str(curr_month.replace(day=1))]

recent_df.loc['Average Loan Size by Status 31-60',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status 31-60',str(curr_month.replace(day=1))] / \
recent_df.loc['Total # of Outstanding Loans by Status 31-60',str(curr_month.replace(day=1))]

recent_df.loc['Average Loan Size by Status 61-90',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status 61-90',str(curr_month.replace(day=1))] / \
recent_df.loc['Total # of Outstanding Loans by Status 61-90',str(curr_month.replace(day=1))]

recent_df.loc['Average Loan Size by Status 91-120',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status 91-120',str(curr_month.replace(day=1))] / \
recent_df.loc['Total # of Outstanding Loans by Status 91-120',str(curr_month.replace(day=1))]

recent_df.loc['Average Loan Size by Status Cumulative 91+',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status Cumulative 91+',str(curr_month.replace(day=1))] / \
recent_df.loc['Total # of Outstanding Loans by Status Cumulative 91+',str(curr_month.replace(day=1))]

recent_df.loc['Average Loan Size by Status Net write-off in the month',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status Net write-off in the month',str(curr_month.replace(day=1))] / \
recent_df.loc['Total # of Outstanding Loans by Status Net write-off in the month',str(curr_month.replace(day=1))]

In [56]:
#Portfolio Month-End Snapshot
Active_users_query = """
select status_date, count(distinct user_reference) from `tensile-oarlock-191715.user_tables.user_status_monthly` where cover_me = 1 group by 1 order by 1;
"""
active_users = pgbq.read_gbq(Active_users_query, project_id, dialect='standard')
active_users = active_users[active_users['status_date'] == (curr_month_end)]

recent_df.loc['Active Cover Users',str(curr_month.replace(day=1))] = active_users.reset_index().loc[0,'f0_']

recent_df.loc['Total Outstanding Balance',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance by Status Current',str(curr_month.replace(day=1))] + \
recent_df.loc['Total Outstanding Balance by Status 1-30',str(curr_month.replace(day=1))] + \
recent_df.loc['Total Outstanding Balance by Status 31-60',str(curr_month.replace(day=1))] + \
recent_df.loc['Total Outstanding Balance by Status 61-90',str(curr_month.replace(day=1))]

recent_df.loc['Total # of Loans Outstanding',str(curr_month.replace(day=1))] = \
recent_df.loc['Total # of Outstanding Loans by Status Current',str(curr_month.replace(day=1))] + \
recent_df.loc['Total # of Outstanding Loans by Status 1-30',str(curr_month.replace(day=1))] + \
recent_df.loc['Total # of Outstanding Loans by Status 31-60',str(curr_month.replace(day=1))] + \
recent_df.loc['Total # of Outstanding Loans by Status 61-90',str(curr_month.replace(day=1))]

recent_df.loc['Avg Balance/Loan',str(curr_month.replace(day=1))] = \
recent_df.loc['Total Outstanding Balance',str(curr_month.replace(day=1))] / \
recent_df.loc['Total # of Loans Outstanding',str(curr_month.replace(day=1))]

recent_df.loc['Total Capital Deployed',str(curr_month.replace(day=1))] = \
recent_df.loc['(Capital Deployed in month) Total Amount of New Loan Issued ',str(curr_month.replace(day=1))] 

Downloading: 100%|██████████|


In [57]:
#Cost Split
recent_df.loc['Cost Split Provision Change',str(curr_month.replace(day=1))] = \
recent_df.loc['Provision Change',str(curr_month.replace(day=1))] 

recent_df.loc['Write Off',str(curr_month.replace(day=1))] = \
recent_df.loc['Write off Change',str(curr_month.replace(day=1))] 

In [58]:
#Loss Rate
recent_df.loc['Write Off Rate (Write Off $ / Outstanding Balance $)',str(curr_month.replace(day=1))] = \
recent_df.loc['Write Off',str(curr_month.replace(day=1))] / \
recent_df.loc['Total Outstanding Balance',str(curr_month.replace(day=1))]

recent_df.loc['Provision Change Rate (Provision Change $ / Outstanding Balance $)',str(curr_month.replace(day=1))] = \
recent_df.loc['Cost Split Provision Change',str(curr_month.replace(day=1))] / \
recent_df.loc['Total Outstanding Balance',str(curr_month.replace(day=1))]

In [59]:
if(forecast_flag):
    forecast_target = format(last_month, '%b %Y') + 'F'
    Forecast_df = Old_Cover_df.loc[:,forecast_target]
    revenue = Forecast_df['Revenue']
    AFA = Forecast_df['AFA']
    
recent_df.loc['Revenue',str(curr_month.replace(day=1))] = revenue
recent_df.loc['AFA',str(curr_month.replace(day=1))] = AFA

recent_df.loc['Cost',str(curr_month.replace(day=1))] = \
recent_df.loc['Cost Split Provision Change',str(curr_month.replace(day=1))] + \
recent_df.loc['Write Off',str(curr_month.replace(day=1))]

recent_df.loc['Net Income',str(curr_month.replace(day=1))] = \
recent_df.loc['Revenue',str(curr_month.replace(day=1))] - \
recent_df.loc['Cost',str(curr_month.replace(day=1))]

recent_df.loc['Average Loan Book',str(curr_month.replace(day=1))] = (\
recent_df.loc['Total Outstanding Balance',str(curr_month.replace(day=1))] + \
recent_df.loc['Total Outstanding Balance',str(last_month.replace(day=1))])/2

recent_df.loc['Return on Capital',str(curr_month.replace(day=1))] = \
recent_df.loc['Net Income',str(curr_month.replace(day=1))] / \
recent_df.loc['Average Loan Book',str(curr_month.replace(day=1))]

recent_df.loc['Annualized Return on Capital',str(curr_month.replace(day=1))] = \
recent_df.loc['Return on Capital',str(curr_month.replace(day=1))] * 12

recent_df.loc['ARPU',str(curr_month.replace(day=1))] = \
recent_df.loc['Revenue',str(curr_month.replace(day=1))] / \
recent_df.loc['AFA',str(curr_month.replace(day=1))]

recent_df.loc['Cost per User',str(curr_month.replace(day=1))] = \
recent_df.loc['Cost',str(curr_month.replace(day=1))] / \
recent_df.loc['AFA',str(curr_month.replace(day=1))]

recent_df.loc['Contribution Margin',str(curr_month.replace(day=1))] = \
recent_df.loc['Net Income',str(curr_month.replace(day=1))] / \
recent_df.loc['AFA',str(curr_month.replace(day=1))]

recent_df.loc['Margin %',str(curr_month.replace(day=1))] = \
recent_df.loc['Net Income',str(curr_month.replace(day=1))] / \
recent_df.loc['Revenue',str(curr_month.replace(day=1))]

In [60]:
recent_df.to_csv("New Cover.csv")

In [65]:
test = True